# Quick Dirty Model with TensorFLow

In [ ]:
# Imports

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
# Reading in data

train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
train.head()

In [ ]:
# Viewing the shape

train.shape

In [ ]:
# Viewing the distribution of target values

train['target'].value_counts()

In [ ]:
# Getting the features column names

features = [col for col in train.columns if col not in ['id', 'target']]

In [ ]:
# Splitting the data into train and validation

X_train, X_valid, y_train, y_valid = train_test_split(
    train[features], 
    train['target'], 
    test_size=0.3, 
    random_state=0
)
X_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
# Creating the model

tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    tf.keras.layers.Dense(
        256, 
        input_shape=(None,100),  
        activation="selu", 
        kernel_initializer='lecun_normal',
    ),
    tf.keras.layers.Dense(
        128, 
        activation="selu",
        kernel_initializer='lecun_normal',
    ),
    tf.keras.layers.Dense(
        128, 
        activation="selu",
        kernel_initializer='lecun_normal',
    ),
    tf.keras.layers.Dense(
        64, 
        activation="selu",
        kernel_initializer='lecun_normal'
    ),
    tf.keras.layers.Dense(
        32, 
        activation="selu",
        kernel_initializer='lecun_normal'
    ),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
# Compiling the model

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(), 'acc']
)

In [ ]:
# Viewing the model

model.summary()

In [ ]:
# # Creating a learning rate scheduler

# def scheduler(epoch, lr):
#     if epoch < epoch/1.5:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# lr_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# Creating early stopping

es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Fitting the model

model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=50,
    callbacks=[es_cb]
)

In [ ]:
# Predicting the on test set
test = scaler.transform(test[features])
preds = model.predict(test)

In [ ]:
# Submitting the predictions

sample_submissions = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sample_submissions['target'] = preds[:,-1]
sample_submissions.to_csv("submission.csv", index=False)
sample_submissions.head()